In [1]:
# 라이브러리 import
import pandas as pd
import numpy as np

from selenium import webdriver  # 라이브러리(모듈) 가져오라
from selenium.webdriver import ActionChains as AC
import chromedriver_autoinstaller
from tqdm import tqdm
from tqdm.notebook import tqdm
import re
from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [4]:
# 데이터 수집할 키워드 지정
keyword = "방역패스"
keyword

'방역패스'

In [5]:
# 크롬창 띄우기
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)
#검색해서 주소창 복붙 후 한글 표시 부분을 지우고 {}.format()으로 변경
driver.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}".format(keyword))
time.sleep(2)

In [8]:
# page1에서 네이버뉴스 url 수집하기
things = driver.find_elements_by_link_text('네이버뉴스')  
things[0].text

'네이버뉴스'

In [9]:
# page1에서 네이버뉴스 url 수집하기
things = driver.find_elements_by_link_text('네이버뉴스')  
things[0].get_attribute("href")

'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=104&oid=005&aid=0001516345'

In [10]:
url_list = []
for thing in things:
    url = thing.get_attribute('href')
    url_list.append(url)

In [11]:
print(len(url_list))
url_list

11


['https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=104&oid=005&aid=0001516345',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=104&oid=469&aid=0000667296',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=018&aid=0005178779',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=104&oid=023&aid=0003682586',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=104&oid=056&aid=0011240117',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=008&aid=0004727219',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=421&aid=0005999242',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=656&aid=0000007224',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=001&aid=0013077168',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=103&oid=277&aid=0005065604',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=110&oid=015&aid=0

In [12]:
df = pd.DataFrame({"url":url_list})
df.to_csv('navernews_urls.csv')

In [29]:
len(df['url'])

11

In [30]:
dict = {}

# n : 크롤링할 뉴스 갯수
n = 10      # 전체 뉴스 수 : len(df['url'])

# 페이지당 기사 수집
for i in tqdm(range(0, n)):   
    try:
        # 뉴스 크롬창 띄우기
        chrome_path=chromedriver_autoinstaller.install()
        driver=webdriver.Chrome(chrome_path)
        driver.get(df['url'][i])
        time.sleep(1)
        # 네이버 뉴스는 iframe 접근이 없음
        # 기사 데이터 수집
        title = driver.find_element_by_css_selector('.tts_head').text
        date = driver.find_element_by_css_selector('.t11').text
        up = int(driver.find_element_by_css_selector('.tomain.as_addinfo .u_cnt._count').text)
        like = int(driver.find_element_by_css_selector(".end_btn .u_likeit_list.good .u_likeit_list_count._count").text)
        warm = int(driver.find_element_by_css_selector(".end_btn .u_likeit_list.warm .u_likeit_list_count._count").text)
        sad = int(driver.find_element_by_css_selector(".end_btn .u_likeit_list.sad .u_likeit_list_count._count").text)
        angry = int(driver.find_element_by_css_selector(".end_btn .u_likeit_list.angry .u_likeit_list_count._count").text)
        want = int(driver.find_element_by_css_selector(".end_btn .u_likeit_list.want .u_likeit_list_count._count").text)

        # 기사 댓글 갯수 
        review_count = int(driver.find_element_by_css_selector(".u_cbox_count").text.replace(',', ''))

        # 더보기 클릭 횟수
        moreview_num = review_count//20

        # 댓글 버튼 클릭
        driver.find_element_by_css_selector(".lo_txt").click()
        time.sleep(1)

        # 더보기 버튼 여러번 클릭하기
        k=0
        while k <= moreview_num:  # 더보기 횟수만큼 반복 
            try:
                driver.find_element_by_css_selector(".u_cbox_page_more").click()  # 더보기 버튼 클릭
                time.sleep(1)
                k = k+1
            except:        
                break   # 에러나면 클릭 반복문을 빠져나가라

        # review 수집하기
        review_list = []
        overlays1 = ".u_cbox_text_wrap"
        reviews = driver.find_elements_by_css_selector(overlays1)
        for review in tqdm(reviews):    
            review = review.text
            review_list.append(review)

        target_info = {}
        target_info['기사명'] = title
        target_info['날짜'] = date
        target_info['기사 추천 수'] = up
        target_info['좋아요'] = like
        target_info['훈훈해요'] = warm
        target_info['슬퍼요'] = sad
        target_info['화나요'] = angry
        target_info['후속기사 원해요'] = want
        target_info['댓글 수'] = len(review_list)
        target_info['댓글'] = review_list

        dict[i] = target_info
        
        print(title, '( 댓글 수:', len(review_list),')')

        driver.close()
        time.sleep(2)
        
    except:
        driver.close()
        continue

print(len(dict))
dict

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

日 언론 “K방역 무너졌다...세계 최다 감염에도 규제 완화” ( 댓글 수: 101 )


  0%|          | 0/67 [00:00<?, ?it/s]

방역패스 중단 한달, 3차 접종 '4분의 1' 토막…오늘부터 어린이 접종 시작 ( 댓글 수: 67 )


...

  0%|          | 0/12 [00:00<?, ?it/s]

"어차피 셀프방역…거리두기 풀든 말든 관심 없어요" ( 댓글 수: 12 )


  0%|          | 0/5 [00:00<?, ?it/s]

‘백신패스 안 본다’ 국내 여행객 무격리 입국 가능국가 총 39개국 ( 댓글 수: 5 )
5


{0: {'기사명': '日 언론 “K방역 무너졌다...세계 최다 감염에도 규제 완화”',
  '날짜': '2022.04.03. 오후 8:09',
  '기사 추천 수': 14,
  '좋아요': 80,
  '훈훈해요': 1,
  '슬퍼요': 4,
  '화나요': 34,
  '후속기사 원해요': 2,
  '댓글 수': 101,
  '댓글': ['문죄앙아라서 그래 맞는말이고 K K K 하면서 까불어댈때 알아봤다. 부르지도 않고 받겨주지도 았는 나라에 정슥이랑 해외 쳐돌아 다닐때 알아봤다',
   '느그나 잘해... 우리 신경끄고..언제부터 관심질이야 ㅋㅋㅋ',
   '글치. 문정부 K방역이 오락가락 개판인건 사실이지. 의료진의 힘과 시스템으로 막았었고 버텼고 사망률최저 타이틀 유지하는게 대단한거지. 대한민국 의료시스템과 의료진의 승리다.',
   '재택치료는 개뿔~~ 재택 방치다~~ 코로나 걸리면 약국에서 약사먹고 배달음식 시켜먹고 알아서 살아남아야한다 이게 지금 우리나라 코로나 방역 현실이다 ㅉㅉㅉ',
   '말은 바로합시다 저팬! K방역이 아니라 주사파 쑈방역시지라~',
   '일본한테 저런 소리 듣는 게 당연할 지경이다.',
   '문죄인이의 거꾸로 방역 끝이없다',
   '조금있음 세계보건기구에서 우리나라가세계최초 엔데믹 발표할거라는데 일본은 🐕소리하네 ㅋㅋㅋㅋㅋ',
   '진심 창피한거다 지들이 그렇게 대놓고 무시한 일본한테 지금 조롱받는게',
   '신경끄라',
   'CNN이랑 WSJ는 높은감염율에도 사망률 낮다고 보도하는데 역시나 일본이 일본하는구나',
   '코로나가 치명적일때 관리잘했고 지금은 완화되었으니 with코로나로 극복해야할 시점',
   '방역포기라고 봐야지',
   '너거나 잘해라 사실방역이 철저하진않았지만 지금 대부분이 걸리는오미크론은 막을래야 막을수가없다ㅠㅠ다행히 우리나라 의료체계가 잘되있고 정부가하라면 어떻게든 해내는 의료계가있기에 이제 점점일상으로 돌아갈수있는 가능성이 보인다고 생각한다',
   '친일정권이 서니 본토에서 내응을 하

In [31]:
# 판다스로 만들기
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df
# result_df.rename(columns={result_df.columns[0] : name}, inplace=True)

,기사명,날짜,기사 추천 수,좋아요,훈훈해요,슬퍼요,화나요,후속기사 원해요,댓글 수,댓글
0,日 언론 “K방역 무너졌다...세계 최다 감염에도 규제 완화”,2022.04.03. 오후 8:09,14,80,1,4,34,2,101,[문죄앙아라서 그래 맞는말이고 K K K 하면서 까불어댈때 알아봤다. 부르지도 않고...
2,"방역패스 중단 한달, 3차 접종 '4분의 1' 토막…오늘부터 어린이 접종 시작",2022.03.31. 오전 10:30,6,0,0,0,130,1,67,"[백신 맞고 후휴증,사망해도 책임도 안지는데 미쳤다고 접종하긋냐?, 이제잘봐라. 주..."
3,英·덴마크 이어 伊도 코로나 방역 풀었다,2022.04.01. 오후 8:55,1,3,0,0,0,1,0,[]
5,"""어차피 셀프방역…거리두기 풀든 말든 관심 없어요""",2022.03.31. 오후 5:20,4,6,2,2,9,0,12,"[작성자에 의해 삭제된 댓글입니다., ㅋㅋ 이따위로 개판쳐놨어도 문재앙과 찢재명한테..."
9,‘백신패스 안 본다’ 국내 여행객 무격리 입국 가능국가 총 39개국,2022.03.29. 오후 2:59,3,24,1,1,2,2,5,"[하는김에 귀국시 격리여부도 표시해주시지... 반쪽짜리 기사네요, 이제 다 풀자 접..."


In [36]:
# 파일로 저장
result_df.to_excel("naver_article_({}).xlsx".format(keyword))
# {}.format(keyword) keyword를 제목으로 하는 파일이 생성 됨